In [ ]:
!python3.11 -m pip install --upgrade --force-reinstall pandas

In [ ]:
!python3.11 -m pip install -q kagglehub ipywidgets 
!python3.11 -m pip install -q scipy

In [ ]:
import pandas as pd 
import ast 

In [ ]:
book_details = pd.read_csv("../data/Book_Details.csv")

In [43]:
book_data = book_details.filter(axis = 'columns', items = ['book_title', 'author', 'book_details', 'num_pages', 'genres'])

In [44]:
def clean_pages(x):
    if x is None:
        return 0
    cleaned = x.strip("[]'")
    if cleaned == 'None' or cleaned == '':
        return 0
    return int(cleaned)

In [45]:
book_data['num_pages'] = book_data['num_pages'].apply(clean_pages)
book_data['genres'] = book_data['genres'].apply(lambda x: ast.literal_eval(x) if x is not None else [])


In [54]:
book_data.dropna(inplace = True)

In [ ]:
import matplotlib.pyplot as plt

# Calculate lengths
lengths = book_data['book_details'].apply(lambda x: len(x) if x is not None else 0)

# Create histogram
plt.figure(figsize=(10, 6))
plt.hist(lengths, bins=30, edgecolor='black')
plt.xlabel('Length of book_details')
plt.ylabel('Frequency')
plt.title('Histogram of book_details Lengths')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
from transformers import BertTokenizer, BertModel 
import torch 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()  # Set to evaluation mode

In [65]:
def get_bert_embedding(text):
    if text is None or text == '':
        return None
    
    # Take first 1000 characters
    text = str(text)[:1000]
    
    # Tokenize and encode
    inputs = tokenizer(text, return_tensors='pt', truncation=True, 
                       max_length=512, padding=True)
    
    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Use [CLS] token embedding - move to CPU first
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    
    return embedding

In [ ]:
!python3.11 -m pip install numpy

In [ ]:
book_data['book_details_embedding'] = book_data['book_details'].apply(get_bert_embedding)